### About this demo

This demo test the time cost between different PennyLane quantum devices.

**Additional packages required:**
- pip install "pennylane-qulacs[cpu]"
- pip install pennylane-lightning

In [ ]:
import os
import sys
import time

import pennylane as qml
import torch
import torch.nn as nn

sys.path.append(os.path.abspath('../'))

from source.models.qcgnn import HybridQCGNN

os.environ['OMP_NUM_THREADS'] = '8'

qml.about()

In [ ]:
q_setup = [
    ('default.qubit', 'best'),
    ('lightning.qubit', 'adjoint'),
    ('qulacs.simulator', 'best'),
]

batch_size = 128
num_ptcs = 16
score_dim = 3

x = torch.rand(batch_size, num_ptcs, 3)
y = torch.randint(low=0, high=score_dim, size=(batch_size, ))

for dev, diff_method in q_setup:

    time_start = time.time()
    
    model = HybridQCGNN(
        num_ir_qubits=4,
        num_nr_qubits=3,
        num_layers=1,
        num_reupload=2,
        num_rotation=1,
        vqc_ansatz=qml.BasicEntanglerLayers,
        score_dim=score_dim,
        qdevice=dev,
        # qbackend='qiskit',
        diff_method=diff_method,
    )
    
    optimizer = torch.optim.RAdam(model.parameters(), lr=1E-2)
    loss_func = nn.CrossEntropyLoss()

    for epoch in range(5):
        optimizer.zero_grad()
        loss = loss_func(model(x), y)
        loss.backward()
        optimizer.step()

        print(f"- Epoch: {epoch} | Loss: {loss.item():.4f}")
    
    time_end = time.time()

    print(f"* Device: {dev} | Time: {time_end - time_start:.2f} s\n")